In [ ]:
! pip install -Uq transformers sentence_transformers datasets fsspec faiss-gpu-cu12

In [ ]:
import wandb
wandb.init(mode="disabled")

### **Load Dataset**

In [ ]:
from datasets import load_dataset

am_relevance_dataset = load_dataset("yosefw/amharic-news-retrieval-dataset-v2-with-negatives-V2")
am_relevance_dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link', 'source_dataset', 'negative_passages'],
        num_rows: 61469
    })
    test: Dataset({
        features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link', 'source_dataset', 'negative_passages'],
        num_rows: 6832
    })
})

In [ ]:
test_passage_ids = set(am_relevance_dataset["test"]["passage_id"])
len(test_passage_ids)

6764

In [ ]:
# Binary dataset

from datasets import Dataset
from tqdm import tqdm
import random

ds_rows = []
for row in tqdm(am_relevance_dataset["train"]):
  ds_rows.append({
      "query_id": row["query_id"],
      "passage_id": row["passage_id"],
      "query": row["query"],
      "passage": row["passage"],
      "label": 1
    })

  neg_passages = row["negative_passages"]

  # Remove negative passages that exist in the test set
  # neg_passages = list(filter(lambda x: x["passage_id"] not in test_passage_ids, neg_passages))

  for neg in neg_passages[:3]+neg_passages[-4:]:
    ds_rows.append({
      "query_id": row["query_id"],
      "passage_id": neg["passage_id"],
      "query": row["query"],
      "passage": neg["passage"],
      "label": 0
    })

# ds_rows = ds_rows[:64000] #
relevance_train_dataset = Dataset.from_list(ds_rows).sort("query_id")#.shuffle(seed=42)
relevance_train_dataset

100%|██████████| 61469/61469 [00:26<00:00, 2333.15it/s]


Dataset({
    features: ['query_id', 'passage_id', 'query', 'passage', 'label'],
    num_rows: 491752
})

In [ ]:
relevance_test_dataset = am_relevance_dataset["test"]#.select(range(100))
relevance_test_dataset

Dataset({
    features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link', 'source_dataset', 'negative_passages'],
    num_rows: 6832
})

In [ ]:
# relevance_train_dataset["query_id"][:10], relevance_train_dataset["passage_id"][:10]

### **Model Training**

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder, CrossEncoderModelCardData
from sentence_transformers.cross_encoder.evaluation import CrossEncoderRerankingEvaluator

# First, we define the transformer model we want to fine-tune
model_name = "rasyosef/roberta-base-amharic"

# We set num_labels=1, which predicts a continuous score between 0 and 1
model = CrossEncoder(
    model_name,
    num_labels=1,
    max_length=510,
    device="cuda",
    model_card_data=CrossEncoderModelCardData(
        language="am",
        license="mit",
        model_name="roberta-amharic-reranker-base",
    ),
  )

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at rasyosef/roberta-base-amharic and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sentence_transformers.util import mine_hard_negatives
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
embedding_model = SentenceTransformer(
    "rasyosef/roberta-amharic-text-embedding-base",
    device="cuda",
  )

hard_eval_dataset = mine_hard_negatives(
        relevance_test_dataset.select_columns(["query", "passage"]),
        embedding_model,
        corpus=am_relevance_dataset["train"]["passage"], #[:8000],  # Use the full dataset as the corpus
        num_negatives=50,  # How many documents to rerank
        batch_size=128,
        include_positives=False,
        output_format="n-tuple",
        use_faiss=True,
    )

print(hard_eval_dataset)

Setting range_max to 52 based on the provided parameters.
Found 6828 unique queries out of 6832 total queries.
Found an average of 1.001 positives per query.


Batches:   0%|          | 0/517 [00:00<?, ?it/s]

Batches:   0%|          | 0/54 [00:00<?, ?it/s]

Querying FAISS index: 100%|██████████| 1/1 [00:00<00:00,  8.29it/s]


Metric       Positive       Negative     Difference
Count           6,832        341,600               
Mean           0.7040         0.5328         0.1713
Median         0.7382         0.5228         0.1890
Std            0.1498         0.0886         0.1491
Min           -0.0702         0.2566        -0.6424
25%            0.6270         0.4710         0.0808
50%            0.7382         0.5228         0.1890
75%            0.8155         0.5850         0.2812
Max            0.9446         0.9402         0.5229
Dataset({
    features: ['query', 'passage', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7', 'negative_8', 'negative_9', 'negative_10', 'negative_11', 'negative_12', 'negative_13', 'negative_14', 'negative_15', 'negative_16', 'negative_17', 'negative_18', 'negative_19', 'negative_20', 'negative_21', 'negative_22', 'negative_23', 'negative_24', 'negative_25', 'negative_26', 'negative_27', 'negative_28', 'negative_29', 'negative

In [ ]:
row = hard_eval_dataset[0]
for col in hard_eval_dataset.column_names:
  print(col,": ", row[col][:64])

query :  የታክስና የጉምሩክ ጉዳዮችን የሚመለከቱ ከ15 በላይ መመርያዎች ይፋ ሊሆኑ ነው
passage :  አዳዲስና የተሻሻሉ ረቂቅ መመርያዎች ሊመከርባቸው ነውየኢትዮጵያ ገቢዎችና ጉምሩክ ባለሥልጣን ከ15 በላ
negative_1 :  ገቢዎች ሚኒስቴርና ገንዘብ ሚኒስቴር ከግብር ጋር የተያያዙ 14 አዳዲስና ነባር መመርያዎችን አሻሽለው 
negative_2 :  በ2012 በጀት ዓመት 248.3 ቢሊዮን ብር ለመሰብሰብ ዕቅድ ይዟል ገቢዎች ሚኒስቴር የኤክሳይስ ታክስ
negative_3 :  አስተላላፊዎች ሥጋት ገብቷቸዋልበጉምሩክ አስተላላፊነት ሙያ መስክ በተሰማሩ ግለሰቦች አሠራርና አደረጃጀ
negative_4 :  ከውጭ አገሮች ለሚገቡ ዕቃዎች ለባለሀብቶችና ለተቋማት ሲሰጥ የቆየው የቀረጥ ነፃ መብት፣ የፖሊሲና የአ
negative_5 :  የፌዴራል የገቢ ግብርና የታክስ አስተዳደር ረቂቅ ደንቦች ከመፅደቃቸው በፊት  ውይይት ሊደረግባቸው ነው
negative_6 :  በየወሩ ሽያጭና ገቢ ማሳወቅን የሚጠይቀውን ነባር አዋጅ እንዲሻሻል ለሕዝብ ተወካዮች ምክር ቤት የቀረበ
negative_7 :  እስከ 3,200 ብር ወርኃዊ ደመወዝ የሚያገኙ ከሚያዝያ ጀምሮ ለሦስት ወራት ከግብር ነፃ ሊሆኑ ይችላሉ
negative_8 :  በቢራና በሲጋራ ዋጋ ላይ 30 እና 40 በመቶ ኤክሳይዝ ታክስ ይጣላልከአሥር ዓመት በላይ ያገለገሉ ተሽ
negative_9 :  የአስቸኳይ ጊዜ አዋጁ ማስፈጸሚያ መመርያ ቁጥር አንድ ሰኞ የካቲት 26 ቀን 2010 ዓ.ም. በፌዴራል 
negative_10 :  የገንዘብና ኢኮኖሚ ትብብር ሚኒስቴር ከገቢ ግብር ነፃ የተደረጉ ገቢዎች አፈጻጸምን የተመለከተውን መመር
negative_11 :  የኢትዮጵያ ብሔራዊ ባንክ ከሁለት ሳምንት በፊት ያረቀቀውና በውጭ ምንዛሪ ንግድ ላይ ግልጽነት 

In [ ]:
# We add an evaluator, which evaluates the performance during training]
dev_evaluator = CrossEncoderRerankingEvaluator(
  samples=[
    {
        "query": row["query"],
        "positive": [row["passage"]],
        "negative": [row[column_name] for column_name in hard_eval_dataset.column_names[2:]],
    }
    for row in hard_eval_dataset.shuffle(seed=42).select(range(1600))
  ],
  at_k=10,
  batch_size=128,
  name="amh-passage-retrieval-dev",
  always_rerank_positives=True,
  show_progress_bar=True
)

In [ ]:
# dev_evaluator(model)

# {'amh-passage-retrieval-dev_map': 0.12662540658833657,
#  'amh-passage-retrieval-dev_mrr@10': 0.09799255952380952,
#  'amh-passage-retrieval-dev_ndcg@10': 0.13502354803772576}

#### **Train**

In [ ]:
import torch
from sentence_transformers.cross_encoder import CrossEncoderTrainer, CrossEncoderTrainingArguments
from sentence_transformers.cross_encoder.losses import BinaryCrossEntropyLoss
from sentence_transformers.training_args import BatchSamplers

run_name = f"roberta-amharic-reranker-base"

num_train_epochs = 4
train_batch_size = 64

args = CrossEncoderTrainingArguments(
    # Required parameter:
    output_dir=f"models/{run_name}",
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=train_batch_size,
    learning_rate=4e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    weight_decay=0.1,
    fp16=True,
    # batch_sampler=BatchSamplers.NO_DUPLICATES,
    dataloader_num_workers=2,
    load_best_model_at_end=True,
    metric_for_best_model="amh-passage-retrieval-dev_mrr@10",
    # Optional tracking/debugging parameters:
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=2,
    # logging_first_step=True,
    run_name=run_name,  # Will be used in W&B if `wandb` is installed
    seed=42,
)

loss = BinaryCrossEntropyLoss(model=model, pos_weight=torch.tensor(7))

# 6. Create the trainer & start training
trainer = CrossEncoderTrainer(
    model=model,
    args=args,
    train_dataset=relevance_train_dataset.select_columns(['query', 'passage', 'label']),
    loss=loss,
    evaluator=dev_evaluator,
)

Token indices sequence length is longer than the specified maximum sequence length for this model (733 > 510). Running this sequence through the model will result in indexing errors


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Amh-passage-retrieval-dev Map,Amh-passage-retrieval-dev Mrr@10,Amh-passage-retrieval-dev Ndcg@10
1,0.348100,No log,0.813071,0.810851,0.847526
2,0.210500,No log,0.842454,0.840460,0.871349
3,0.122100,No log,0.857847,0.856127,0.883872
4,0.055500,No log,0.853470,0.852914,0.881509


TrainOutput(global_step=30736, training_loss=0.18406562666170675, metrics={'train_runtime': 9430.6285, 'train_samples_per_second': 208.577, 'train_steps_per_second': 3.259, 'total_flos': 0.0, 'train_loss': 0.18406562666170675, 'epoch': 4.0})

In [ ]:
# Save latest model
model.save_pretrained(run_name + "-latest")

# **Evaluation**

In [ ]:
# We add an evaluator, which evaluates the performance during training]
evaluator = CrossEncoderRerankingEvaluator(
  samples=[
    {
        "query": row["query"],
        "positive": [row["passage"]],
        "negative": [row[column_name] for column_name in hard_eval_dataset.column_names[2:]],
    }
    for row in hard_eval_dataset
  ],
  at_k=10,
  batch_size=128,
  name="amh-passage-retrieval-dev",
  always_rerank_positives=True,
  show_progress_bar=True
)

evaluator(model)

{'amh-passage-retrieval-dev_map': 0.8531100005859653,
 'amh-passage-retrieval-dev_mrr@10': 0.8513254037953979,
 'amh-passage-retrieval-dev_ndcg@10': 0.8802377937215004}

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

# model = CrossEncoder("rasyosef/roberta-amharic-reranker-medium")
scores = model.predict([
    ["በለንደን የዩክሬን ደጋፊ አገራት መሪዎች ጉባኤ", "ጀርመን አገር የተማሩ ኢትዮጵያዉያን ቁጥር ከአፍሪቃ አንደኛ ነዉ። እነ ካሜሩን ከነኬንያን ሁሉ ይበልጣል። በማኅበር የመደራጀታችን ዋናዉ ዓላማ የኢትዮ-ጀርመንን ግንኙነትን ከፍ ማድረግ ነዉ። ከ 10 ሺህ በላይ ኢትዮጵያዉያን ተማሪዎች በጀርመን አገር ተምረዉ ተመልሰዋል። አልያም አዉሮጳ ወደ አሜሪካም የተሻገሩ አሉ።"],
    ["በለንደን የዩክሬን ደጋፊ አገራት መሪዎች ጉባኤ", "በጋምቤላ እስካሁን 23 ሰዎች በኮሌራ መሞታቸውን የተናገሩት አንድ የጤና ባለሙያ በሽታውን ለመከላከል አንድ የግል ተቋም ድጋፍ ማድረጉን እና በዓለም ጤና ድርጅት የህክምና ድጋፍ መሰጠቱን አብራርተዋል፡፡ ከደቡብ ሱዳን አጎራባች አኮቦ ወረዳ ለመጀመሪያ ጊዜ መከሰቱ በተነገረው nበሽታው ብዙ ሰዎች የተጠቁት ዋንቱዋ በተባለ ወረዳ መሆኑን ጠቁመዋል፡፡"],
    ["በለንደን የዩክሬን ደጋፊ አገራት መሪዎች ጉባኤ", "በደም እጥረት ምክንያት ለህሙማን አስፈላጊዉን ህክምና ለመስጠት ተቸግረናል ሲሉ የተናገሩት የቆቦ ሆስፒታል ስራ አስኪያጅ ቢኖሩም የደም እጥረቱ ሀኪሞች አገልግሎት እንዲሰጡ አላስቻላቸዉም ይላሉ። የወልድያ ደም ባንክ አገልግሎት ስራ አስኪያጅ አሁን ለአስቸኳይ ህክምና ብቻ የሚሆን ለ20 ቀናት ብቻ የሚቆይ ደም ነዉ ያለን ይላሉ"],
    ["በለንደን የዩክሬን ደጋፊ አገራት መሪዎች ጉባኤ", "አሜሪካ ለውጭ የምትሰጠውን እርዳታ ማቆሟ በተለይ በአፍሪቃ ሃገራት ከፍተኛ ግርታና ድንጋጤ ፈጥሯል። ለሁለተኛ ጊዜ ወደ የሥልጣን መንበር የመጡት የአሜሪካው ፕሬዝደንት ዶናልድ ትራምፕ የውጭ እርዳታ ለዘጠና ቀናት እንዲቆም ወስነዋል። ትራምፕ እርዳታ አፍሪቃውያንን አሳንፏቸዋል ነው የሚሉት።"],
    ["በለንደን የዩክሬን ደጋፊ አገራት መሪዎች ጉባኤ", "በለንደን የብሪታኒያው ጠ/ሚኒ ኬይር ስታርመር አስተናጋጅነት የተካሄደው ዋናዎቹ የዩክሬን ደጋፊ አገሮች መሪዎች ጉባኤ ዩክሬንን በወታደራዊና ፋይናናስ የበለጠ ለመርዳት ተስማምቶ ማብቃቱ ተገልጿል። ይህ ከአውሮፓ ህብረት ውጭ የካናዳንና የኖርዌይ መሪዎች ብሎም የቱርክን የውጭ ጉዳይ ሚኒስትር ጉባኤ የተጠራዉ የነጬ ቤተመንግስት ክስተትን ተከትሎ ነዉ።"]
  ])
scores

array([4.8988205e-03, 6.4008392e-04, 1.0056645e-03, 1.8900981e-03,
       9.9966955e-01], dtype=float32)

In [ ]:
from google.colab import userdata

model.push_to_hub("RoBERTa-Amharic-Reranker-Base", exist_ok=True, token=userdata.get("HF_WRITE"))